# Agentic AI Workshop — Part 1: Basic LLM Calls, Streaming, and Chat

This notebook demonstrates:
- Text-in → Text-out with the OpenAI Python SDK
- Text → Image generation
- Text → Audio (TTS)
- A minimal Gradio UI for an LLM call
- Streaming responses
- A simple multi‑turn chat

> **Prereqs**
> - `pip install openai gradio`  
> - Set your API key in the environment as `OPENAI_API_KEY` (recommended).


In [1]:
import os, io, base64, time
from openai import OpenAI
import gradio as gr

# Load environment variables from .env file
import sys
sys.path.append('../utils')
from helpers import load_environment, get_env_var

# Load environment variables
load_environment()
print("Environment variables loaded successfully!")


Environment variables loaded successfully!


In [2]:
# Verify API key is loaded
api_key = get_env_var('OPENAI_API_KEY')
if not api_key or api_key == 'your_openai_api_key_here':
    print("⚠️  Please set your OPENAI_API_KEY in the .env file")
else:
    print("✅ OpenAI API key found!")


⚠️  Please set your OPENAI_API_KEY in the .env file


In [ ]:
# Expect OPENAI_API_KEY to be in env; you can also set it explicitly here (not recommended).
# os.environ['OPENAI_API_KEY'] = "sk-..."

client = OpenAI()  # Uses OPENAI_API_KEY from environment
print("Client ready.")


In [ ]:
# --- 1) Basic text generation
prompt = "Write a friendly, two-sentence definition of Retrieval-Augmented Generation."
resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}]
)
print(resp.choices[0].message.content)


In [ ]:
# --- 2) Image generation (Text → Image)
# Saves a PNG called 'workshop_image.png'.
img = client.images.generate(
    model="dall-e-3",
    prompt="Flat illustration of a helpful robot librarian fetching knowledge from a bookshelf labeled 'RAG'",
    size="1024x1024"
)
b64 = img.data[0].b64_json
png_bytes = base64.b64decode(b64)
with open("workshop_image.png", "wb") as f:
    f.write(png_bytes)

print("Saved image to workshop_image.png")


In [ ]:
# --- 3) Audio generation (Text → Speech)
# Saves an MP3 called 'workshop_audio.mp3'.
# The 'voice' options may vary; try 'alloy', 'verse', or others supported by your account.
speech = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input="Welcome to the Agentic AI workshop! Today we will build with RAG and LLM workflows."
)

with open("workshop_audio.mp3", "wb") as f:
    f.write(speech.content)

print("Saved audio to workshop_audio.mp3")


In [ ]:
# --- 4) Gradio UI: single-turn LLM call

def generate_reply(user_text):
    r = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=[{"role": "user", "content": user_text}]
    )
    return r.choices[0].message.content

demo = gr.Interface(
    fn=generate_reply,
    inputs=gr.Textbox(label="Ask the model"),
    outputs=gr.Textbox(label="Model reply"),
    title="Basic LLM Call",
    description="Single-turn text → text using OpenAI Chat Completions API."
)

# Uncomment to launch locally
# demo.launch()


In [ ]:
# --- 5) Streaming example (console print)

print("Streaming tokens:")
stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Explain RAG in one paragraph."}],
    stream=True
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)

print("\n\n[done]")


In [ ]:
# --- 6) Multi-turn Chat with Gradio

SYSTEM_PROMPT = "You are a concise teaching assistant helping users learn Agentic AI fundamentals."

def chat_bot(history, user_msg):
    # Convert Gradio history to messages
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    for turn in history:
        user, assistant = turn
        if user:
            messages.append({"role": "user", "content": user})
        if assistant:
            messages.append({"role": "assistant", "content": assistant})
    messages.append({"role": "user", "content": user_msg})

    r = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7
    )
    reply = r.choices[0].message.content
    history.append((user_msg, reply))
    return history, ""

with gr.Blocks(title="Multi‑Turn Chat") as chat_app:
    gr.Markdown("### Multi‑Turn Chat (Gradio)")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Ask something about agentic AI...")
    clear = gr.Button("Clear")

    def clear_fn():
        return []

    msg.submit(chat_bot, [chatbot, msg], [chatbot, msg])
    clear.click(clear_fn, [], [chatbot])

# Uncomment to launch locally
# chat_app.launch()
